In [1]:
import os
import pandas as pd
import numpy as np
import scipy as sp
from scipy import signal
#from ml_time_series import generate_envelope
import ml_time_series as mls
import timeit
from sklearn.externals import joblib
from ml_time_series_theano import generate_envelope_nd_T

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: unknown error)


---
# Processing
---

In [10]:
import numpy as np
import theano
from theano import tensor as T

def generate_envelope_nd_TT(original_data, num_samples):
    # initialize
    new_Xc = np.array([])
    x = T.dmatrix('x')
    w = theano.shared(num_samples-1)
    update = [(w, w + 1)]
    envelope_nd = x[w-(num_samples-1):w+1][0 : num_samples, :].ravel()
    
    # compile model
    net_input = theano.function(inputs=[x],
                                updates=update,
                                outputs=envelope_nd)
    # execute
    X_input =  np.array((original_data), 
                          dtype=theano.config.floatX)

    if num_samples < len(original_data):
        new_Xc = net_input(X_input) 
        for i in np.arange(num_samples, original_data.shape[0]):
            new_Xc = np.vstack((new_Xc, net_input(X_input)))

    else:
        print 'num_samples must be lower than original_data.'
    return new_Xc

In [11]:
def proccess_data(FILE_,DATA,NUM_SAMPLES,LABEL):

    Xc = np.genfromtxt('DB_GAIT/'+FILE_+'.txt', delimiter=",", usecols=(1,2,3,4))
    
    print '\nXc shape ', Xc.shape
    
    #Create temporal serie
    #Xc = mls.generate_envelope(Xc, NUM_SAMPLES)
    Xc = generate_envelope_nd_TT(Xc, NUM_SAMPLES)
    print 'Xc temporal-serie shape ', Xc.shape
    
    #Labeling the type of movement
    C = (np.ones(len(Xc))*LABEL).reshape((len(Xc),1))
    Xc = np.hstack((Xc.reshape(Xc.shape),C.reshape((len(Xc),1))))
    print 'Xc labeled shape ', Xc.shape
    
    np.save('./preprocessed_data/'+FILE_, Xc, allow_pickle=False)
    print FILE_+'.npy'
    
    DATA.append(FILE_+'.npy')

In [12]:
def window_nd(a, window, steps = None, axis = None, outlist = False):
    """
    Create a windowed view over `n`-dimensional input that uses an 
    `m`-dimensional window, with `m <= n`

    Parameters
    -------------
    a : Array-like
        The array to create the view on

    window : tuple or int
        If int, the size of the window in `axis`, or in all dimensions if 
        `axis == None`

        If tuple, the shape of the desired window.  `window.size` must be:
            equal to `len(axis)` if `axis != None`, else 
            equal to `len(a.shape)`, or 
            1

    steps : tuple, int or None
        The offset between consecutive windows in desired dimension
        If None, offset is one in all dimensions
        If int, the offset for all windows over `axis`
        If tuple, the steps along each `axis`.  
            `len(steps)` must me equal to `len(axis)`

    axis : tuple, int or None
        The axes over which to apply the window
        If None, apply over all dimensions
        if tuple or int, the dimensions over which to apply the window

    outlist : boolean
        If output should be as list of windows.  
        If False, it will be an array with 
            `a.nidim + 1 <= a_view.ndim <= a.ndim *2`.  
        If True, output is a list of arrays with `a_view[0].ndim = a.ndim`
            Warning: this is a memory-intensive copy and not a view

    Returns
    -------

    a_view : ndarray
        A windowed view on the input array `a`, or copied list of windows   

    """
    ashp = np.array(a.shape)

    if axis != None:
        axs = np.array(axis).reshape(-1)
        assert np.all(np.in1d(axs, np.arange(ashp.size))), "Axes out of range"
    else:
        axs = np.arange(ashp.size)

    window = np.array(window).reshape(-1)
    assert (window.size == axs.size) | (window.size == 1), "Window dims and axes don't match"
    wshp = ashp.copy()
    wshp[axs] = window
    assert np.all(wshp <= ashp), "Window is bigger than input array in axes"

    stp = np.ones_like(ashp)
    if steps:
        steps = np.array(steps).reshape(-1)
        assert np.all(steps > 0), "Only positive steps allowed"
        assert (steps.size == axs.size) | (steps.size == 1), "Steps and axes don't match"
        stp[axs] = steps

    astr = np.array(a.strides)

    shape = tuple((ashp - wshp) // stp + 1) + tuple(wshp)
    strides = tuple(astr * stp) + tuple(astr)

    as_strided = np.lib.stride_tricks.as_strided
    a_view = np.squeeze(as_strided(a, 
                                 shape = shape, 
                                 strides = strides))
    if outlist:
        return list(a_view.reshape((-1,) + tuple(wshp)))
    else:
        return a_view

In [14]:
NUM_SAMPLES = 500

DATA = []

start = timeit.default_timer()
proccess_data('data_w_1',DATA,NUM_SAMPLES,0)

stop = timeit.default_timer()

ti = stop - start 
print ('Tempo total para o envelopamento: %.2f min' % (ti/60))


Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_1.npy
Tempo total para o envelopamento: 0.61 min


In [13]:
files_w = ['data_w_1', 'data_w_2', 'data_w_3', 'data_w_4', 'data_w_5', 'data_w_6', 'data_w_7', 'data_w_8', 'data_w_9', 
           'data_w_10', 'data_w_11', 'data_w_12']
files_r = ['data_r_1', 'data_r_2', 'data_r_3', 'data_r_4', 'data_r_5', 'data_r_6', 'data_r_7', 'data_r_8', 'data_r_9', 
           'data_r_10', 'data_r_11', 'data_r_12']
files_us = ['data_us_1', 'data_us_2', 'data_us_3', 'data_us_4', 'data_us_5', 'data_us_6', 'data_us_7', 'data_us_8', 
            'data_us_9', 'data_us_10', 'data_us_11', 'data_us_12']
files_ds = ['data_ds_1', 'data_ds_2', 'data_ds_3', 'data_ds_4', 'data_ds_5', 'data_ds_6', 'data_ds_7', 'data_ds_8', 
            'data_ds_9', 'data_ds_10', 'data_ds_11', 'data_ds_12']

NUM_SAMPLES = 500

DATA = []

start = timeit.default_timer()

for i in files_w:
    proccess_data(i,DATA,NUM_SAMPLES,0)

for i in files_r:
    proccess_data(i,DATA,NUM_SAMPLES,1)

for i in files_us:
    proccess_data(i,DATA,NUM_SAMPLES,2)

for i in files_ds:
    proccess_data(i,DATA,NUM_SAMPLES,3)

print DATA

stop = timeit.default_timer()

ti = stop - start 
print ('Tempo total para o envelopamento: %.2f min' % (ti/60))


Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_1.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_2.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_3.npy

Xc shape  (6001, 4)
Xc temporal-serie shape  (5502, 2000)
Xc labeled shape  (5502, 2001)
data_w_4.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_5.npy

Xc shape  (6001, 4)
Xc temporal-serie shape  (5502, 2000)
Xc labeled shape  (5502, 2001)
data_w_6.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_7.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_8.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled shape  (4502, 2001)
data_w_9.npy

Xc shape  (5001, 4)
Xc temporal-serie shape  (4502, 2000)
Xc labeled sha

In [22]:
%%bash
paplay /usr/share/sounds/freedesktop/stereo/complete.oga

['data_w_1.npy', 'data_w_2.npy', 'data_w_3.npy', 'data_w_4.npy', 'data_w_5.npy', 'data_w_6.npy', 'data_w_7.npy', 'data_w_8.npy', 'data_w_9.npy', 'data_w_10.npy', 'data_w_11.npy', 'data_w_12.npy', 'data_r_1.npy', 'data_r_2.npy', 'data_r_3.npy', 'data_r_4.npy', 'data_r_5.npy', 'data_r_6.npy', 'data_r_7.npy', 'data_r_8.npy', 'data_r_9.npy', 'data_r_10.npy', 'data_r_11.npy', 'data_r_12.npy', 'data_us_1.npy', 'data_us_2.npy', 'data_us_3.npy', 'data_us_4.npy', 'data_us_5.npy', 'data_us_6.npy', 'data_us_7.npy', 'data_us_8.npy', 'data_us_9.npy', 'data_us_10.npy', 'data_us_11.npy', 'data_us_12.npy', 'data_ds_1.npy', 'data_ds_2.npy', 'data_ds_3.npy', 'data_ds_4.npy', 'data_ds_5.npy', 'data_ds_6.npy', 'data_ds_7.npy', 'data_ds_8.npy', 'data_ds_9.npy', 'data_ds_10.npy', 'data_ds_11.npy', 'data_ds_12.npy']

In [2]:
DATA = ['data_w_1.npy', 'data_w_2.npy', 'data_w_3.npy', 'data_w_4.npy', 'data_w_5.npy', 'data_w_6.npy', 
        'data_w_7.npy',  
        'data_r_1.npy', 'data_r_2.npy', 'data_r_3.npy', 'data_r_4.npy', 'data_r_5.npy', 'data_r_6.npy', 
        'data_r_7.npy', 
        'data_us_1.npy', 'data_us_2.npy', 'data_us_3.npy', 'data_us_4.npy', 'data_us_5.npy', 'data_us_6.npy', 
        'data_us_7.npy',  
        'data_ds_1.npy', 'data_ds_2.npy', 'data_ds_3.npy', 'data_ds_4.npy', 'data_ds_5.npy', 'data_ds_6.npy', 
        'data_ds_7.npy']

len(DATA)

28

In [3]:
start = timeit.default_timer()

Xc = np.load("preprocessed_data/"+DATA[0])
for i in DATA[1:]:
    Xc = np.vstack((Xc,np.load("preprocessed_data/"+i))) 
    
stop = timeit.default_timer()
ti = stop - start 
print ('Tempo total para o envelopamento: %.2f min' % (ti/60))
Xc.shape

Tempo total para o envelopamento: 0.20 min


(134056, 2001)

save_it_now(Xc, './preprocessed_data/vstack1.data')

start = timeit.default_timer()

for i in DATA[30:]:
    Xc = np.vstack((Xc,load_file("preprocessed_data/"+i))) 
    
stop = timeit.default_timer()
ti = stop - start 
print ('Tempo total para o envelopamento: %.2f min' % (ti/60))

Xc.shape

In [4]:
Xc.shape

(134056, 2001)

In [5]:
X = Xc[:,0:(Xc.shape[1]-2)]

yz = Xc[:,[(Xc.shape[1]-1)]]
y = np.array([])
for i in range(len(yz)):
    y = np.hstack((y,yz[i]))

X.shape, y.shape

((134056, 1999), (134056,))

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    random_state=10)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [8]:
print X_train_std.shape

(93839, 1999)


# Random Forest Classifier

In [9]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, n_jobs = 4)

print 'Training time'
%time rfc = rfc.fit(X_train_std, y_train)

print''

print 'Predict time'
%time y_pred = rfc.predict(X_test_std)

print''

from sklearn.metrics import accuracy_score
print ('ClassifyRF accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))

Training time
CPU times: user 11min 49s, sys: 972 ms, total: 11min 50s
Wall time: 3min 1s

Predict time
CPU times: user 6.58 s, sys: 120 ms, total: 6.7 s
Wall time: 1.83 s

ClassifyRF accuracy:---------->95.92 %


In [10]:
#Save the classifier to file
joblib.dump(rfc, 'rfc.pkl') 


['rfc.pkl']

In [7]:
#Load the classifier from file
rfc = joblib.load('rfc.pkl')

## Validation

In [34]:
DATAV = ['data_w_9.npy', 'data_r_9.npy', 'data_us_9.npy', 'data_ds_9.npy']

Xc = np.load("preprocessed_data/"+DATAV[0])
for i in DATAV[1:]:
    Xc = np.vstack((Xc,np.load("preprocessed_data/"+i)))
    
Xvalid = Xc[:,0:(Xc.shape[1]-2)]

yz = Xc[:,[(Xc.shape[1]-1)]]
yvalid = np.array([])
for i in range(len(yz)):
    yvalid = np.hstack((yvalid,yz[i]))

print Xvalid.shape, yvalid.shape

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#sc.fit(Xvalid)
sc.fit(X_train)
X_valid_std = sc.transform(Xvalid)

print X_valid_std.shape

(18008, 1999) (18008,)
(18008, 1999)


In [35]:
#Validation
print 'Predict time'
%time y_pred = rfc.predict(X_valid_std)

print''

from sklearn.metrics import accuracy_score

result = ('ClassifyRF accuracy:---------->%.2f %%' % (accuracy_score(y_pred, yvalid)*100))

print result

Predict time
CPU times: user 1.94 s, sys: 76 ms, total: 2.01 s
Wall time: 741 ms

ClassifyRF accuracy:---------->36.00 %


ClassifyRF 8  accuracy:---------->36.00 %
ClassifyRF 9  accuracy:---------->50.07 %
ClassifyRF 10 accuracy:---------->60.37 %
ClassifyRF 11 accuracy:---------->59.10 %
ClassifyRF 12 accuracy:---------->50.10 %